# Select variables from the CESM1LE dataset to compute ensemble average

### Handle multiple time frames and correct for weird CESM time convention

Use bash cdo and nco (ncea).

In [1]:
# Where ensemble outputs are stored by variable
cesm1le_vardir="/glade/campaign/cesm/collections/cesmLE/CESM-CAM5-BGC-LE/atm/proc/tseries/monthly"

In [2]:
# scratch for work
scratch_dir="/glade/scratch/jonahshaw/"
# scratch for processed data
scratch_savedir="/glade/scratch/jonahshaw/processed/"

In [3]:
ls $scratch_savedir

b.e11.B20TRC5CNBDRD.f09_g16.001-035_101-105.cam.h0.FLUTC.192001-200512.nc
b.e11.B20TRC5CNBDRD.f09_g16.001-035_101-105.cam.h0.FSNS.192001-200512.nc
b.e11.B20TRC5CNBDRD.f09_g16.001-035_101-105.cam.h0.FSNSC.192001-200512.nc
b.e11.B20TRC5CNBDRD.f09_g16.001-035_101-105.cam.h0.FSNTOAC.192001-200512.nc
b.e11.BRCP85C5CNBDRD.f09_g16.001-035_101-105.cam.h0.FLUTC.200601-208012.nc
b.e11.BRCP85C5CNBDRD.f09_g16.001-035_101-105.cam.h0.FSNS.200601-208012.nc
b.e11.BRCP85C5CNBDRD.f09_g16.001-035_101-105.cam.h0.FSNSC.200601-208012.nc
b.e11.BRCP85C5CNBDRD.f09_g16.001-035_101-105.cam.h0.FSNTOAC.200601-208012.nc


In [4]:
# Variables to retreive
# cesm1le_vars=("ICEFRAC" "TS" "SOLIN" "FSNTOA" "FLUT" "TMQ")
# cesm1le_vars="ICEFRAC TS SOLIN FSNTOA FSNTOAC FLUT FLUTC TMQ FSNS FSNSC"
# cesm1le_vars="FSNTOAC FLUTC FSNS FSNSC"
cesm1le_vars="FLNS FLDS"

In [5]:
module load cdo
module load nco

In [6]:
# iterate over variables of interest
for var in $cesm1le_vars
do 
    echo $var
    cd $cesm1le_vardir/$var/
    filepaths1=$(ls *BRCP85C5CNBDRD*200601*.nc)
#     filepaths=$(ls $cesm1le_vardir/$var/*BRCP85C5CNBDRD*200601*.nc)
#     echo $filepaths
    for path in $(echo $filepaths1)
    do
        echo $path
        
        cdo shifttime,-1mo "$cesm1le_vardir/$var/$path" $scratch_dir/$path
        
#         break
    done
    
    # iterate over files with a different time period and select 2006-2080
    filepaths2=$(ls *BRCP85C5CNBDRD*200601-210012*.nc)
    for path in $(echo $filepaths2)
    do
        echo $path
        
        cdo selyear,2006/2080 "$scratch_dir/$path" "$scratch_dir/$path"b
                
    done
    
    ls $scratch_dir*.ncb
    ls $scratch_dir/*BRCP85C5CNBDRD*200601-208012*.nc
    
    # compute ensemble average of corrected files and save to processed/ directory
    ncea $scratch_dir/*$var*.ncb $scratch_dir/*BRCP85C5CNBDRD*$var*200601-208012*.nc $scratch_savedir/b.e11.BRCP85C5CNBDRD.f09_g16.001-035_101-105.cam.h0.$var.200601-208012.nc
    
    # clean up scratch
    rm $scratch_dir/*$var*.nc
    rm $scratch_dir/*$var*.ncb
#     break
done 


FLNS
b.e11.BRCP85C5CNBDRD.f09_g16.001.cam.h0.FLNS.200601-208012.nc
Warning (cdf_check_variables): Unsupported data type (char/string), skipped variable date_written!
Warning (cdf_check_variables): Unsupported data type (char/string), skipped variable time_written!
cdo    shifttime: Processed 49777067 values from 16 variables over 900 timesteps [2.25s 75MB].
b.e11.BRCP85C5CNBDRD.f09_g16.002.cam.h0.FLNS.200601-208012.nc
Warning (cdf_check_variables): Unsupported data type (char/string), skipped variable date_written!
Warning (cdf_check_variables): Unsupported data type (char/string), skipped variable time_written!
cdo    shifttime: Processed 49777067 values from 16 variables over 900 timesteps [2.34s 75MB].
b.e11.BRCP85C5CNBDRD.f09_g16.003.cam.h0.FLNS.200601-208012.nc
Warning (cdf_check_variables): Unsupported data type (char/string), skipped variable date_written!
Warning (cdf_check_variables): Unsupported data type (char/string), skipped variable time_written!
cdo    shifttime: Process

: 1